<a href="https://colab.research.google.com/github/dominiksakic/zero_to_hero/blob/main/adv_05_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal
Train a GPT to do addition of two numbers, i.e. a+b=c. You may find it helpful to predict the digits of c in reverse order, as the typical addition algorithm (that you're hoping it learns) would proceed right to left too. You may want to modify the data loader to simply serve random problems and skip the generation of train.bin, val.bin. You may want to mask out the loss at the input positions of a+b that just specify the problem using y=-1 in the targets (see CrossEntropyLoss ignore_index). Does your Transformer learn to add? Once you have this, swole doge project: build a calculator clone in GPT, for all of +-*/. Not an easy problem. You may need Chain of Thought traces.

- Where to get a dataset? Create on yourself? [Yourself]
  - write a script that randomized two numbers a and b.
  - loop over n steps and record the results in a text file.
- How to tokenize the input?
  - Char level?
  - Tokens?
  - See below for some ideas.
- Train a simple GPT on the dataset like you would on any other text. What are the outputs? Why?
- How would you train in reverse order?

# Try 1
- Simplest version of a calculator 0 - 9 + 0 - 9 = 00 - 18
- one data set would be blocksize 6.
- how much data could I create? The number of combinations?
  - 81 combinations only? That is not a lot and I would think that the neural net can just remember these!
  - Due to the lack of data the network should be able to remember all the data making a perfect calculator.
- I just have to mask the last to blocks.

In [37]:
import random

# create Dataset
text = ''
for i in range(10):
  for j in range(10):
    if (i+j) < 10:
      result = f"{i}+{j}=0{i+j}"
    else:
      result = f"{i}+{j}={i+j}"

    text += str(result)

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

def encode(s):
    return [stoi[c] for c in s]

def decode(l):
    return ''.join(itos[i] for i in l)

In [38]:
chars

['+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=']

In [39]:
encode("9+9=18")

[10, 0, 10, 11, 2, 9]

In [66]:
# try a simple network takes in a integer tensor and outputs a integer tensor

import torch

block_size = 4
target_size = 2

data = encode(text)

X, Y = [], []

for i in range(0, (len(data) - block_size), 6):
    context = data[i : i + block_size]
    target  = data[i + block_size : i + block_size + target_size]

    # print(decode(context), "->", decode(target))

    X.append(context)
    Y.append(target)

X = torch.tensor(X)  # shape (N, 4)
Y = torch.tensor(Y)  # shape (N, 2)

X.shape

torch.Size([100, 4])